# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

In [ ]:
def clean_columns(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ","_")
    df.columns = df.columns.str.replace(r'\bst\b', 'state', regex=True)
    return df.columns

df.columns = clean_columns(df)

In [ ]:
# 1
df2 = df[(df.total_claim_amount > 1000) & (df.response=="Yes")]

In [75]:
# 2
df[(df.response=="Yes")].groupby(["policy_type","gender"])['total_claim_amount'].mean()

policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64

The average total claim amount of female on corporate auto and special auto are higher than male, while for personal auto policy type there is no significant different across gender.

In [72]:
# 3
# Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
df3 = df.drop_duplicates(subset=["customer","state"])
state = df.groupby(["state"])['customer'].count()
state = state[state>500]
state.sort_values(ascending=False)

state
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: customer, dtype: int64

In [89]:
# 4 
#Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
df.groupby(['education', 'gender']).agg({"customer_lifetime_value":['max','min','median']}).round()

customer_lifetime_value                
                                                max     min    mean
education            gender                                        
Bachelor             F                      73226.0  1904.0  7874.0
                     M                      67907.0  1898.0  7704.0
College              F                      61850.0  1899.0  7749.0
                     M                      61135.0  1918.0  8052.0
Doctor               F                      44856.0  2396.0  7329.0
                     M                      32677.0  2268.0  7415.0
High School or Below F                      55277.0  2145.0  8675.0
                     M                      83325.0  1941.0  8150.0
Master               F                      51016.0  2418.0  8157.0
                     M                      50568.0  2272.0  8169.0

Disparities of customer lifetime value median between gender within the same education level only appears on customer with college education level, with the discrepancy around 400.

In [125]:
#5
# The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.
df["month"] = df["effective_to_date"].str.split("/").str[0]
df.pivot_table(index='state', columns='month', values=['number_of_policies'], aggfunc='sum')

number_of_policies      
month                       1     2
state                              
Arizona                  3052  2864
California               5673  4929
Nevada                   1493  1278
Oregon                   4697  3969
Washington               1358  1225

In [ ]:
#6
#Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

In [145]:
df["month"] = df["effective_to_date"].str.split("/").str[0]
df6 = df.pivot_table(index='state', columns='month', values=['number_of_policies'], aggfunc='sum')
df6["number_of_policies","Total"]=df6["number_of_policies","1"]+df6["number_of_policies","2"]
df6.sort_values([("number_of_policies","Total")], ascending=False).head(3)

number_of_policies             
month                       1     2  Total
state                                     
California               5673  4929  10602
Oregon                   4697  3969   8666
Arizona                  3052  2864   5916

In [146]:
#7
#The marketing team wants to analyze the effect of different marketing channels on the customer response rate.
df['response'] = df['response'].apply(lambda x: 1 if x=='Yes' else 0)
melted_table = pd.melt(df, id_vars=['sales_channel'], value_vars=['response'])
response_rate_table = melted_table.groupby('sales_channel')['value'].sum().reset_index()
response_rate_table

sales_channel  value
0         Agent    742
1        Branch    326
2   Call Center    221
3           Web    177